### Rate - признак давления/скорости течения газов
Если подумать то есть некое базовое смещение ~180 получасовок + rate влияет на это смещение в +/-


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go

In [2]:
from constants import FEATURE_COLUMNS, FEATURE_GASES, TARGET_COLUMNS, FEATURE_GASES_MASS, TARGET_COLUMNS_MASS

from load_data import load_data
from processing import process
from modelling import evaluate_training, plot_fitted_values
from submission import create_submission
from evaluate import predict
from plotting import plot_submition

In [3]:
train_features, train_targets, test_features = load_data()

train_features, train_targets, test_features = process(
        (train_features, train_targets, test_features))

train_data = pd.concat([train_targets, train_features], axis=1)

In [4]:
train_data.head()

,B_C2H6,B_C3H8,B_iC4H10,B_nC4H10,A_rate,B_rate,A_CH4,A_C2H6,A_C3H8,A_iC4H10,...,A_nC5H12,A_C6H14,A_CH4_specified,A_C2H6_specified,A_C3H8_specified,A_iC4H10_specified,A_nC4H10_specified,A_iC5H12_specified,A_nC5H12_specified,A_C6H14_specified
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-01-01 00:00:00,4.561979,39.015486,17.250740,24.301793,70.855017,71.982789,0.126244,5.242957,41.94148,17.064213,...,4.062079,4.11017,0.124266,5.160814,41.284372,16.796864,22.842380,4.119283,3.998438,4.045775
2020-01-01 00:30:00,4.556172,39.032758,17.239769,24.278888,70.855017,71.973649,0.126244,5.242957,41.94148,17.064213,...,4.062079,4.11017,0.124282,5.161469,41.289615,16.798997,22.845281,4.119806,3.998946,4.046289
2020-01-01 01:00:00,4.538603,38.937981,17.215523,24.263735,70.855017,71.935849,0.126244,5.242957,41.94148,17.064213,...,4.062079,4.11017,0.124347,5.164181,41.311311,16.807824,22.857285,4.121971,4.001047,4.048415
2020-01-01 01:30:00,4.551377,38.974181,17.216349,24.232353,70.855017,71.909199,0.126244,5.242957,41.94148,17.064213,...,4.062079,4.11017,0.124394,5.166095,41.326622,16.814054,22.865756,4.123498,4.002530,4.049915
2020-01-01 02:00:00,4.564143,38.938550,17.191849,24.190752,70.855017,71.778992,0.126244,5.242957,41.94148,17.064213,...,4.062079,4.11017,0.124619,5.175467,41.401588,16.844554,22.907235,4.130978,4.009790,4.057262


In [5]:
for gas in ['C2H6','C3H8','iC4H10','nC4H10']:
    fig = go.Figure()
    b_gas = f"B_{gas}"
    a_gas = f"A_{gas}"
    fig.add_trace(go.Scatter(x=train_data.index, y=train_data[b_gas], name=b_gas))
    fig.add_trace(go.Scatter(x=train_data.index, y=train_data[a_gas], name=a_gas))
    fig.show()

In [6]:
train_data.corr().style.background_gradient(cmap='RdBu')

,B_C2H6,B_C3H8,B_iC4H10,B_nC4H10,A_rate,B_rate,A_CH4,A_C2H6,A_C3H8,A_iC4H10,A_nC4H10,A_iC5H12,A_nC5H12,A_C6H14,A_CH4_specified,A_C2H6_specified,A_C3H8_specified,A_iC4H10_specified,A_nC4H10_specified,A_iC5H12_specified,A_nC5H12_specified,A_C6H14_specified
B_C2H6,1.000000,0.213194,-0.657887,-0.776702,-0.208754,0.407289,0.304263,0.976976,0.233406,-0.786060,-0.782937,0.068599,-0.139840,0.128197,0.241258,0.921148,-0.379724,-0.694837,-0.703468,-0.260683,-0.406382,-0.204184
B_C3H8,0.213194,1.000000,0.116225,-0.383844,0.165520,0.313718,-0.293964,0.200360,0.796116,-0.098584,-0.386822,-0.556637,-0.631383,-0.492318,-0.312212,0.192919,0.211307,-0.138438,-0.253032,-0.494668,-0.510913,-0.438127
B_iC4H10,-0.657887,0.116225,1.000000,0.773450,0.164835,-0.144470,-0.422619,-0.680940,-0.088018,0.906691,0.726096,-0.570620,-0.441292,-0.229383,-0.399411,-0.700409,0.201566,0.534757,0.496154,-0.271030,-0.137756,-0.028034
B_nC4H10,-0.776702,-0.383844,0.773450,1.000000,0.127165,-0.372110,-0.353595,-0.784976,-0.497932,0.881845,0.965775,-0.230094,-0.084348,-0.025994,-0.304763,-0.768966,0.185237,0.658868,0.715299,0.086632,0.208055,0.218960
A_rate,-0.208754,0.165520,0.164835,0.127165,1.000000,0.189870,-0.338172,-0.179506,0.183857,0.145703,0.144440,-0.251875,-0.146292,-0.243828,-0.205196,0.042951,0.699997,0.561016,0.519321,0.231460,0.311704,0.207762
B_rate,0.407289,0.313718,-0.144470,-0.372110,0.189870,1.000000,-0.168004,0.458430,0.399459,-0.280433,-0.396025,-0.220228,-0.280073,-0.303311,-0.276704,0.281782,-0.464038,-0.626195,-0.645007,-0.602871,-0.629313,-0.628504
A_CH4,0.304263,-0.293964,-0.422619,-0.353595,-0.338172,-0.168004,1.000000,0.297508,-0.139847,-0.473810,-0.390713,0.360017,0.378211,0.445933,0.978752,0.309258,-0.195920,-0.292359,-0.273915,0.191180,0.179004,0.263935
A_C2H6,0.976976,0.200360,-0.680940,-0.784976,-0.179506,0.458430,0.297508,1.000000,0.304360,-0.805735,-0.813035,0.057249,-0.155606,0.017707,0.228992,0.940493,-0.368310,-0.716847,-0.729810,-0.283635,-0.432523,-0.298924
A_C3H8,0.233406,0.796116,-0.088018,-0.497932,0.183857,0.399459,-0.139847,0.304360,1.000000,-0.269066,-0.576112,-0.514347,-0.570054,-0.692063,-0.172623,0.290874,0.240478,-0.246192,-0.374001,-0.505262,-0.513930,-0.621661
A_iC4H10,-0.786060,-0.098584,0.906691,0.881845,0.145703,-0.280433,-0.473810,-0.805735,-0.269066,1.000000,0.885942,-0.427809,-0.296729,-0.196408,-0.438359,-0.809155,0.220682,0.648901,0.636405,-0.102172,0.021428,0.053677
